# Monthly Daycon Flight Delay Prediction AI Competition 

In [ ]:
#안녕하세요. 데이콘입니다.
#
#이번 대회는 LogLoss를 통해 모델의 성능을 평가하기에,
#predict가 아닌 predict_proba를 이용해서 추론을 진행했습니다.
#
#데이콘 베이스라인은 모든 참가자의 '제출'을 목표로 합니다. 
#해당 내용에 관한 질문은 [토크] 게시판에 게시해 주세요.
#
#베이스라인이 여러분만의 고도화된 모델 작성에 도움이 되면 좋겠습니다.
#
#감사합니다. 
#데이콘 드림

'''
-train.csv [파일]
1,000,000개의 데이터
ID : 샘플 고유 id
항공편 운항 관련 정보
Month: 해당 항공편의 출발 월
Day_of_Month: Month에 해당하는 월의 날짜
Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간 (현지 시각, HH:MM 형식)
Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간 (현지 시각, HH:MM 형식)
Cancelled: 해당 항공편의 취소 여부 (0: 취소되지 않음, 1: 취소됨)
Diverted: 해당 항공편의 경유 여부 (0: 취소되지 않음, 1: 취소됨)
Origin_Airport: 해당 항공편 출발 공항의 고유 코드 (IATA 공항 코드)
Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
Destination_Airport: 해당 항공편 도착 공항의 고유 코드 (IATA 공항 코드)
Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
Airline: 해당 항공편을 운항하는 항공사
Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드 
(IATA 공항 코드, 단 다른 항공사가 같은 코드를 보유할 수도 있음)
Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
Delay: 항공편 지연 여부 (Not_Delayed, Delayed)
예측해야 하는 타깃
다수의 데이터에 레이블이 존재하지 않음
준지도학습을 통해 레이블링 가능
'''

In [ ]:
'''
test.csv [파일]
1,000,000개의 데이터
ID : 샘플 고유 id
항공편 운항 관련 정보
Month: 해당 항공편의 출발 월
Day_of_Month: Month에 해당하는 월의 날짜
Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간 (현지 시각, HH:MM 형식)
Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간 (현지 시각, HH:MM 형식)
Cancelled: 해당 항공편의 취소 여부 (0: 취소되지 않음, 1: 취소됨)
Diverted: 해당 항공편의 경유 여부 (0: 취소되지 않음, 1: 취소됨)
Origin_Airport: 해당 항공편 출발 공항의 고유 코드 (IATA 공항 코드)
Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
Destination_Airport: 해당 항공편 도착 공항의 고유 코드 (IATA 공항 코드)
Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
Airline: 해당 항공편을 운항하는 항공사
Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드 
(IATA 공항 코드, 단 다른 항공사가 같은 코드를 보유할 수도 있음)
Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호

- sample_submission.csv [제출양식]
ID : 샘플 고유 id
Not_Delayed: 특정 샘플에 해당하는 항공편이 지연되지 않을 확률
Delayed: 특정 샘플에 해당하는 항공편이 지연될 확률
'''

## data pre-processing

#### function import

In [1]:
import numpy as np
import pandas as pd
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf
from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer


## Baseline 코드

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)
seed_everything(42) #Seed 고정

## csv to parquet 
- 메모리에 효율적인 데이터 유형을 사용하여 용량을 줄이고 빠른 작업이 가능.

In [ ]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
csv_to_parquet('./train.csv', 'train' )
csv_to_parquet('./test.csv', 'test')

# train done
# tewt done

## Data Load

In [ ]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

## Data Pre-Processing

In [ ]:
# 레이블(Delay)을 제외한 결측값이 존재하는 변수들을 학습 데이터의 최빈값으로 대체함
NaN_col = ['Origin_State', 'Destination_State', 'Airline', 'Estimated_Departure_Time', 'Estimated_Arrival_Time', 'Carrier_Code(IATA)', 'Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

In [ ]:
#질적 변수들을 수치화
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

In [ ]:
#레이블이 없는 데이터들을 제거합니다
train = train.dropna()

In [ ]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

In [ ]:
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

## Classification Model Fit

In [ ]:
clf = RandomForestClassifier()
clf.fit(train_x, train_y)

## Inference

In [ ]:
y_pred = clf.predict_proba(test_x)

## Submit

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)

submission.to_csv('baseline_submission.csv', index=True)

#### Load Data

In [3]:
### 학습용 데이터 불러오기

file_path = 'D:\Code\월간 데이콘 항공편 지연 예측 AI 경진대회/train.csv'

df_train = pd.read_csv(file_path)

print(f'학습용 데이터 : \n{df_train}')

print('*'*80)

### 평가용 데이터 불러오기

file_path = 'D:\Code\월간 데이콘 항공편 지연 예측 AI 경진대회/test.csv'

df_test = pd.read_csv(file_path)

print(f'평가용 데이터 : \n{df_test}')

학습용 데이터 : 
                  ID  Month  Day_of_Month  Estimated_Departure_Time  \
0       TRAIN_000000      4            15                       NaN   
1       TRAIN_000001      8            15                     740.0   
2       TRAIN_000002      9             6                    1610.0   
3       TRAIN_000003      7            10                     905.0   
4       TRAIN_000004      1            11                     900.0   
...              ...    ...           ...                       ...   
999995  TRAIN_999995      9            18                     936.0   
999996  TRAIN_999996      5            30                     920.0   
999997  TRAIN_999997      6            28                     800.0   
999998  TRAIN_999998      9            27                    1613.0   
999999  TRAIN_999999      3            26                    1800.0   

        Estimated_Arrival_Time  Cancelled  Diverted Origin_Airport  \
0                          NaN          0         0            OKC

#### 컬럼의 설명 주석
- 'Estimated_Departure_Time' : 예상 출발 시간
- 'Estimated_Arrival_Time' : 예상 도착 시간
- 'Diverted' : 회항(방향전환)

In [4]:
columns = df_train.columns

print(columns) 


Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number', 'Delay'],
      dtype='object')


#### 학습용 데이터의 결측치 확인 및 제거

In [5]:
n_nulls = df_train.isnull().sum()

print(n_nulls)

print('*'*80)

# 8개의 컬럼을 제거

print(df_train)

print('*'*80)

cleaned_df_train = df_train.drop(columns=['Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Airline','Carrier_Code(IATA)','Carrier_ID(DOT)','Delay'])

print(cleaned_df_train)

# 1. df_train.dropna(subset=[
## subset의 axis는 어치파 인 지워도 됨. 누락값을 지울거니까. 컬럼만 지정해줄 때 쓰는 subset.
# 2. Delay가 정답컬럼과 연관이 있는지..? 

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64
********************************************************************************
                  ID  Month  Day_of_Month  Estimated_Departure_Time  \
0       TRAIN_000000      4            15                       NaN   
1       TRAIN_000001      8            15                     740.0   
2       TRAIN_000002      9 